In [1]:
import decouple

In [2]:
config = decouple.AutoConfig(' ')

In [3]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [4]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [5]:
ALPHA_VANTAGE_API_KEY

'4DA3RF1H7L46HH9X'

In [6]:
import requests

In [7]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
params = {
    "api_key": ALPHA_VANTAGE_API_KEY,
    "function": "TIME_SERIES_INTRADAY",
    "ticker": "AAPL",
    "interval": "5min"
}

In [8]:
url = "https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={api_key}".format(
    **params
)

In [9]:
r = requests.get(url)

In [10]:
data = r.json()

In [11]:
# data

In [12]:
data.keys()

dict_keys(['Meta Data', 'Time Series (5min)'])

In [13]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]

In [14]:
dataset_key

'Time Series (5min)'

In [15]:
results = data[dataset_key]

In [16]:
# results[0]

In [17]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2025-01-15 19:55:00'

In [18]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '238.6000',
 '2. high': '238.6000',
 '3. low': '238.4500',
 '4. close': '238.5000',
 '5. volume': '2099'}

In [19]:
from datetime import datetime
import pytz
from decimal import Decimal

def transform_alphavantage_result(timestamp_str, result):
    timestamp_format = "%Y-%m-%d %H:%M:%S"
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    
    timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    timestamp
    return {
        "volume": int(result["5. volume"]),
        "open": Decimal(result["1. open"]),
        "close": Decimal(result["4. close"]),
        "high": Decimal(result["2. high"]),
        "low": Decimal(result["3. low"]),
        "timestamp": timestamp,  
    }

In [20]:
transform_alphavantage_result(timestamp_str, result_val)

{'volume': 2099,
 'open': Decimal('238.6000'),
 'close': Decimal('238.5000'),
 'high': Decimal('238.6000'),
 'low': Decimal('238.4500'),
 'timestamp': datetime.datetime(2025, 1, 16, 0, 55, tzinfo=<UTC>)}

In [22]:
for timestamp in results.keys():
    new_data = transform_alphavantage_result(timestamp, results.get(timestamp))
    # print(new_data)